## Reconocimiento de basurales en imagen satelital

Nota:
    
Las bandas de color de los raster a analizar deben estar en el orden estándar : RGB, Red-Green-Blue / Rojo-Verde-Azul 

Las imágenes descargadas de Copernicus vienen en otro orden orden (Azul, Verde, Rojo) por lo que si se utiliza input tomado de esa fuente se deben reorganizar las bandas antes de procesar.

Las imágenes descargadas vía Google Earth Engine estan listas para usar.


### Preprocesamiento de imagen: generación de mosaicos / chips

In [ ]:
# Ubicación de la(s) imagen(es) a procesar

image_location = "/tmp/GEE/*"

In [ ]:
# area de interés (la zona a analizar)

aoi_file = "../data/aoi/RMBA/RMBA_envolvente.shp"

In [ ]:
chip_dir = "/tmp/chips/"     # Directorio donde dejar los resultados

chip_size = 160                 # tamaño de los "chips" (recortes cuadrados) en píxeles por lado

In [ ]:
!satproc_extract_chips \
    $image_location \
    -o $chip_dir \
    --aoi $aoi_file \
    --size $chip_size \
    --step-size $chip_size

### Detección

In [ ]:
from unetseg.predict import PredictConfig, predict
from unetseg.evaluate import plot_data_generator, plot_data_results

In [ ]:
# Ubicación del modelo
model_file = "../modelos/modelo_deteccion_basura.h5"

In [ ]:
# Si necesitamos descargarlo

#url_modelo = "https://basurales.s3.amazonaws.com/AMBA/modelos_CIM/basurales_AMBA_2023_04_chipsize_160_48.h5"
#!wget {url_modelo} -O $model_file

#### Parámetros para el modelo

In [ ]:
# batch size: cantidad de datos que se procesan por vez, puede ser limitado por 
# la memoria de gpu disponible 
batch = 96

# tamaño de la imagen procesada por la UNet (debe ser multiplos de 16 , por ej 160, 320, etc) 
# y lo mas parecida posible a la resolucion de los chips
unet_input_size = chip_size

# cantidad de clases a detectar
clases = 1

# Directorio donde dejar los resultados
resultados = "/tmp/chips_predicciones/"


predict_config = PredictConfig(images_path = chip_dir,
                               results_path = resultados, 
                               batch_size = batch,
                               model_path = model_file,
                               height = unet_input_size,
                               width = unet_input_size,
                               n_channels = 3, # porque trabajamos con imagen a color, 3 canales: Red, Green, Blue
                               n_classes = clases)


#### Iniciamos el proceso de reconocimiento

In [ ]:
predict(predict_config)

#### Visualizamos algunos resultados

In [ ]:
plot_data_results(num_samples=5, predict_config=predict_config)

### Exportar las predicciones 

In [ ]:
from satproc.postprocess.polygonize import polygonize 
from satproc.filter import filter_by_max_prob
from pathlib import Path

In [ ]:
# Donde vamos a guardar las predicciones

export_dir = "../predicciones/"

#### Postprocesamiento

In [ ]:
# 1. Definimos un umbral de corte, descartando los pixeles con probabilidad menor

thresh = 0.6  # valor del umbral (o "threshold en inglés"): 60% de probabilidad o más de corresponder a un basural

input_path_unfiltered= "/tmp/chips_predicciones/"
output_path_filtered = "/tmp/chips_predicciones_filtered/" + str(thresh).replace(".", "") + "/"


filter_by_max_prob(input_dir=input_path_unfiltered,
                   output_dir=output_path_filtered,
                   threshold=thresh)

# 2. Generamos polígonos cubriendo las áreas de los pixeles que superan el umbral

input_path = output_path_filtered
output_path = export_dir + 'prediccion_basurales_umbral_' + str(thresh).replace(".", "") + '.gpkg'

polygonize(threshold=thresh,
           input_dir=input_path,
           output=output_path)

In [ ]:
# Aquí quedan los resultados:

!realpath $output_path